# RAG Retrieval Optimization - Query Rewriting using Amazon Bedrock and Llamaindex

Query rewrite in RAG is a technique that reformulates or transforms the original user query to improve the retrieval process and ultimately enhance the quality of generated responses. This strategy involves modifying the input query in various ways, such as expanding it with related terms, simplifying complex queries, or breaking them down into sub-questions. The goal is to bridge the gap between the user's natural language input and the system's ability to find relevant information in the knowledge base. By rewriting queries, RAG systems can increase both recall (retrieving more relevant documents) and precision (improving the relevance of retrieved information), leading to more accurate and comprehensive answers.

In this lab, we will build a query engine using LlamaIndex to answering a complex query about Amazon's 10K SEC filing from years 2022 and 2023. The lab uses the SubQuestionQueryEngine module from LLamaIndex to first breaks down the complex query into sub questions for each relevant data source then gathers all the intermediate responses and synthesizes to get better final response from Amazon's 10K documents.

- Vector Database (Faiss / local)
- LLM (Amazon Bedrock - Claude3 Sonnet)
- Embeddings Model (Bedrock Titan Text Embeddings v2.0)
- Datasets ( Amazons 10-k sec filings from year 2022 and 2023 )
- Llamaindex  (This example is built on referece llamaindex documentation available at - https://docs.llamaindex.ai/en/stable/examples/query_engine/sub_question_query_engine/)


### > Setup
We start by importing necessary llamaindex libraries

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core import Settings

We select Anthropic Claude3 Sonnet as our LLM. For embedding model, we are selecting Amazon Titan Text Embed v2.0. Chunk size is set at 256 for this example.

In [ ]:
import json
from typing import Sequence, List
from llama_index.core.settings import Settings
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding, Models

llm = Bedrock(model = "anthropic.claude-3-sonnet-20240229-v1:0")
embed_model = BedrockEmbedding(model = "amazon.titan-embed-text-v2:0")

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
import nest_asyncio
nest_asyncio.apply()

Using the LlamaDebugHandler to print the trace of the sub questions captured by the SUB_QUESTION callback event type

In [ ]:

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

Settings.callback_manager = callback_manager

### > Document Ingestion
We ingest the documents and use [Titan Text Embeddings v2.0 model](https://docs.aws.amazon.com/bedrock/latest/userguide/titan-embedding-models.html) to create the embedding for each document chunk. The amazon folder has SEC-10k files from 2022 and 2023.

In [ ]:
# load data
amazon_secfiles = SimpleDirectoryReader(input_dir="../data/lab03/amazon/").load_data()

# build index and query engine
vector_query_engine = VectorStoreIndex.from_documents(
    amazon_secfiles,
    use_async=True,
).as_query_engine()

SubQuestionQueryEngine will use the LLM model (Claude3 Sonnet) to breaking down of complex queries into sub-queries.

In [ ]:
# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="Amazon-10k",
            description="Amazon SEC 10-k filings for years 2022 and 2023",
        ),
    ),
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=True,
)

### > Test Question One

In [ ]:
response = query_engine.query(
    "What were key challenges faced by Amazon in year 2022 and 2023?"
)

### > Final response

In [ ]:
print(response)

### > Programmatically iterate through SUB_QUESTION event

In [ ]:
# iterate through sub_question items captured in SUB_QUESTION event
from llama_index.core.callbacks import CBEventType, EventPayload

for i, (start_event, end_event) in enumerate(
    llama_debug.get_event_pairs(CBEventType.SUB_QUESTION)
):
    qa_pair = end_event.payload[EventPayload.SUB_QUESTION]
    print("Sub Question " + str(i) + ": " + qa_pair.sub_q.sub_question.strip())
    print("Answer: " + qa_pair.answer.strip())
    print("====================================")

### > Test Question 2

In [ ]:
response = query_engine.query(
    "What are amazons key priorities before, during and after covid?"
)

### > Final response

In [ ]:
print(response)